## Resting State Transfer Entropy Differentiates Responders and Non-Responders to Oral Ketamine Treatment for Suicidality

Code created by Jules Mitchell March 2024.

You are free to use this or any other code from this repository for your own projects and publications. Citation or reference to the repository is not required, but would be much appreciated (see more on README.md).

In [ ]:
# Import packages
import pandas as pd
import os 
import pickle
import numpy as np
from pathlib import Path

# Import classes
from idtxl.multivariate_te import MultivariateTE
from idtxl.data import Data
from idtxl.visualise_graph import plot_network, plot_selected_vars, plot_network_comparison
from idtxl.stats import network_fdr
from idtxl import idtxl_io as io

## Choose estimator to import
# import idtxl.estimators_python # Non-linear continous data Kraskov estimator 
import idtxl.estimators_jidt

# Set directories
os.chdir('C:/Users/j_m289/Pictures/phd/3. Data Analysis/studies/OKTOS/resting_te/data')
root_dir = os.getcwd()
output_dir = '/derivatives'

In [ ]:
def load_bids(root_folder, classifier):
    """
    Load BIDS formatted data from a root folder.

    Parameters:
    - root_folder (str or Path): Path to the root folder containing BIDS data.
    - classifier (list): List of subjects classified as responder.

    Returns:
    - bids_files (list): List of dictionaries containing BIDS data information.
    """
    root_folder = Path(root_folder)
    bids_files = []

    # Walk through all subject folders in the root directory
    for subject_folder in root_folder.iterdir():
        if not subject_folder.is_dir():
            continue

        # Check if subject is a responder or non-responder
        if subject_folder.name in classifier:
            response_category = "responder"
        else:
            response_category = "non_responder"

        # Walk through all ses-0* folders in the current subject folder
        for session_folder in subject_folder.glob('ses-*'):
            if not session_folder.is_dir():
                continue

            # Check for 'eeg' folder and load CSV files
            eeg_path = session_folder / 'eeg'
            if not eeg_path.exists():
                files = [f for f in session_folder.glob('*.csv') if f.is_file()]
            else:
                # Assuming graph files are stored as CSV files (modify as needed)
                files = [f for f in eeg_path.glob('*.csv') if f.is_file()]

            # Load each CSV file into a directed graph
            for file_path in files:
                df = pd.read_csv(file_path, header=None)

                # Drop channels from dataframe and select portion of resting state data
                df = df.drop(columns=0)
                df = df.iloc[:, 0:100]  # Modify as per your requirements

                # Save dataframe as IDTxl data object and specify dimensions
                data = Data(df, dim_order='ps')

                # Determine task based on filename or path
                if 'EC' in file_path.stem:
                    task = 'EC'
                else:
                    task = 'EO'

                # Add the graph information to the list
                bids_files.append({
                    "subject": subject_folder.name,
                    "response_category": response_category,
                    "session": session_folder.name,
                    "task": task,
                    "data": data
                })

    return bids_files

def combine_targets(folder, threshold=2, contains='EC', output_path='derivatives'):
    """
    Combines targets from files in specified folders and saves the results as pickle files.

    Parameters:
    - folder (str or Path): Path to the root folder containing subject and session folders.
    - threshold (int): Threshold value used in network_fdr function.
    - contains (str): Substring to filter files by.
    - output_path (str or Path): Path where the output pickle files will be saved.

    Returns:
    - res: Result from network_fdr function.
    """
    # Convert folder and output_path to Path objects if they are strings
    folder = Path(folder)
    output_path = Path(output_path)

    # Initialize res as None to handle case where no files are processed
    res = None

    # Loop through subject folders
    for subject_folder in folder.iterdir():
        if not subject_folder.is_dir():
            continue
        
        if subject_folder.name == 'derivatives':  # Compare against 'name' attribute
            continue

        # Loop through session folders
        for session_folder in subject_folder.iterdir():
            if not session_folder.is_dir():
                continue

            # Filter files based on whether the file name contains the 'contains' substring
            files = [f for f in session_folder.iterdir() if f.is_file() and contains in f.name]

            # Skip processing if there are no files to combine
            if not files:
                print(f'No files containing "{contains}" found in {session_folder}')
                continue

            # Load results using pickle
            res_list = []

            for file in files:
                with open(file, 'rb') as f:
                    res_list.append(pickle.load(f))

            # Perform operation to combine targets
            res = network_fdr({'alpha_fdr': 0.05, 'fdr_constant': threshold}, *res_list)

            # Construct the output file path using pathlib
            output_dir = folder / output_path / subject_folder.name / session_folder.name
            output_dir.mkdir(parents=True, exist_ok=True)  # Create the directory if it doesn't exist
            output_file = f'{subject_folder.name}_{session_folder.name}_{contains}_network.p'
            file_path = output_dir / output_file
            
            # Save combined results as pickle file
            with open(file_path, 'wb') as f:
                pickle.dump(res, f)

            print(f'Successfully Combined {len(res_list)} Targets for {subject_folder.name}/{session_folder.name}')
    return res 

## Combine Targets

In [ ]:
# Eyes closed
single_targets = 'C:/Users/j_m289/Pictures/phd/3. Data Analysis/studies/OKTOS/resting_te/data'
res = combine_targets(single_targets, threshold=1, contains='EC') # Threshold set to 1 to be less stringent (i.e. less false negatives, but more potential false positives)

In [ ]:
# Eyes open
single_targets = 'C:/Users/j_m289/Pictures/phd/3. Data Analysis/studies/OKTOS/resting_te/data'
res = combine_targets(single_targets, threshold=1, contains='EO') # Threshold set to 1 to be less stringent (i.e. less false negatives, but more potential false positives)

### Visual check 

In [ ]:
# Load individual network files to view directed graphs
#os.chdir('C:/Users/j_m289/Pictures/phd/3. Data Analysis/studies/OKTOS/resting_te/data/bids')
file = 'C:/Users/j_m289/Pictures/phd/3. Data Analysis/studies/OKTOS/resting_te/data/derivatives/sub-40/ses-01/sub-40_ses-01_EC_network.p'
with open(file, 'rb') as f:
	res = pickle.load(f)

plot_network(results=res, weights='max_te_lag', fdr=True)